In [63]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import pandas as pd
import datetime as dt

In [65]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [4]:
??proc_df

In [66]:
path = "data/sapflow/"

In [90]:
weather_csv = pd.read_csv(f'{path}all_weather.txt', 
                          low_memory=False, parse_dates=[["Date","Time"]], encoding='ISO-8859-1', index_col=False)

sapflow_csv = pd.read_csv(f'{path}all_sapflow.txt', 
                          low_memory=False, parse_dates=[["Date","Time"]], encoding='ISO-8859-1', index_col=False)

In [91]:
weather_csv["Date_Time"] = weather_csv["Date_Time"].dt.strftime('%d/%m/%Y %H:%M')
sapflow_csv["Date_Time"] = sapflow_csv["Date_Time"].dt.strftime('%d/%m/%Y %H:%M')

In [92]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(weather_csv)

In [93]:
weather_csv = weather_csv[["Date_Time", "Solar Radiation (W/m^2)", "Air Temperature (deg.C)", "Corrected RH (%)", "Wind Speed (m/s)"]]
weather_csv.rename(columns = {"Date_Time":"datetime", "Solar Radiation (W/m^2)":"solar_rad","Air Temperature (deg.C)":"air_temp","Corrected RH (%)":"RH","Wind Speed (m/s)":"wind_speed"}, inplace=True)

In [94]:
sapflow_csv = sapflow_csv[["Date_Time", "Uncorrected Out (cm/hr)", "Uncorrected In (cm/hr)"]]
sapflow_csv.rename(columns ={"Date_Time":"datetime", "Uncorrected Out (cm/hr)":"outer_sapflow", "Uncorrected In (cm/hr)":"inner_sapflow"}, inplace=True)

In [95]:
merged=pd.merge(weather_csv, sapflow_csv, on='datetime', how='inner')

merged=merged.drop_duplicates()

In [ ]:
display_all(merged)

In [96]:
def label_time (row):
    hour = dt.datetime.strptime(row['datetime'],'%d/%m/%Y %H:%M').hour
    if hour >= 6 and hour < 12:
        return 'morning'
    if hour >= 12 and hour < 17:
        return 'afternoon'
    if hour >= 17 and hour < 20:
        return 'evening'
    if hour >= 20 and hour <= 24:
        return 'night'
    if hour >= 0 and hour < 6:
        return 'night'

In [ ]:
#merged.apply (label_time, axis=1)

In [97]:
merged['time_of_day'] = merged.apply (label_time, axis=1)

In [ ]:
display_all(merged)

In [98]:
df_outer = merged.drop("inner_sapflow", axis=1)
df_inner = merged.drop("outer_sapflow", axis=1)

In [99]:
df_outer = df_outer[pd.to_numeric(df_outer['outer_sapflow'], errors='coerce').notnull()]
df_inner = df_inner[pd.to_numeric(df_inner['inner_sapflow'], errors='coerce').notnull()]

In [100]:
df_outer['outer_sapflow'] = pd.to_numeric(df_outer['outer_sapflow'])
df_inner['inner_sapflow'] = pd.to_numeric(df_inner['inner_sapflow'])

In [101]:
outer_min = df_outer['outer_sapflow'].min()
inner_min = df_inner['inner_sapflow'].min()

In [102]:
df_outer['new_sapflow'] = df_outer.apply(lambda row: row['outer_sapflow'] + abs(outer_min), axis=1)
df_inner['new_sapflow'] = df_inner.apply(lambda row: row['inner_sapflow'] + abs(inner_min), axis=1)

In [103]:
display_all(df_outer.isnull().sum().sort_index()/len(df_outer))

RH               0.000036
air_temp         0.045428
datetime         0.000000
new_sapflow      0.000000
outer_sapflow    0.000000
solar_rad        0.045392
time_of_day      0.000000
wind_speed       0.045428
dtype: float64

Use to save csv files and temp

In [104]:
merged.to_csv(f'{path}merged.csv', index =False)
df_outer.to_csv(f'{path}df_outer.csv', index =False)
df_inner.to_csv(f'{path}df_inner.csv', index =False)

In [ ]:
df_outer.read_csv()

In [108]:
os.makedirs('tmp', exist_ok=True)
df_outer.reset_index(inplace=True)
df_inner.reset_index(inplace=True)
df_outer.to_feather('tmp/df_outer-raw')
df_inner.to_feather('tmp/df_inner-raw')
#del df_outer[index]
#del df_inner[index]
display_all(df_outer)

ValueError: cannot insert level_0, already exists

# df_outer

In [109]:
df_outer = pd.read_feather('tmp/df_outer-raw')

In [110]:
display_all(df_outer)

,level_0,index,datetime,solar_rad,air_temp,RH,wind_speed,outer_sapflow,time_of_day,new_sapflow
0,0,0,21/03/2018 19:50,0.0,-0.100,44.559,1.839,-3.620,evening,16.018
1,1,1,21/03/2018 20:00,0.0,-0.400,45.840,1.570,-3.435,night,16.203
2,2,2,21/03/2018 20:10,0.0,-0.500,46.479,1.179,-3.077,night,16.561
3,3,3,21/03/2018 20:20,0.0,-0.600,47.360,1.660,-1.704,night,17.934
4,4,4,21/03/2018 20:30,0.0,-0.800,48.319,1.820,0.038,night,19.676
5,5,5,21/03/2018 20:40,0.0,-0.800,48.560,2.319,0.998,night,20.636
6,6,6,21/03/2018 20:50,0.0,-1.000,49.680,1.679,0.678,night,20.316
7,7,7,21/03/2018 21:00,0.0,-1.200,50.479,1.750,0.214,night,19.852
8,8,8,21/03/2018 21:10,0.0,-1.299,51.599,1.660,-0.156,night,19.482
9,9,9,21/03/2018 21:20,0.0,-1.500,52.319,1.500,-0.449,night,19.189


In [26]:
train_cats(df_outer)

In [27]:
df_outer.time_of_day.cat.categories

Index(['afternoon', 'evening', 'morning', 'night'], dtype='object')

In [29]:
df_outer.time_of_day.cat.set_categories(['morning', 'afternoon', 'evening', 'night'], ordered=True, inplace=True)

In [30]:
df_outer.time_of_day = df_outer.time_of_day.cat.codes

In [59]:
df_trn, y_trn, nas = proc_df(df_outer, 'new_sapflow')

In [60]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 3000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_outer, n_trn)

In [61]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [62]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.05774683324755075, 1.1671352865812727, 0.9999415760721216, 0.970445405220799, 0.9997955800642458]


In [ ]:
??